In [ ]:

import pandas as pd
import numpy as np




<h2 id="DADOS-AUSENTES:">DADOS AUSENTES:<a class="anchor-link" href="#DADOS-AUSENTES:">¶</a></h2>



<p>Nessa seção vamos aprender como lidar com tabelas incompletas. Por padrão o pandas reconhece apenas alguns caracteres como valores ausentes (que seria 'NA', 'NaN' ou simplesmente um vazio).</p>
<p>Mas as vezes um processo pode marcar um valor ausente com algum caracter específico e o pandas consegue trabalhar muito bem nessas situações bastante apenas utilizar a option 'na_values' da ação de leitura '.read_csv()' como no exemplo abaixo:</p>


In [ ]:

data = pd.read_csv('/Users/julianaforlin/Downloads/vehicles/vehicles_messy.csv',low_memory=False)
data.info()



<p>Se ainda assim existir alguma coluna que tenha algum caso bem específico como, por exemplo, uma determinada variável só pode ter letras mas por algum motivo veio um número inteiro no meio. Pode realizar o seguinte comando:</p>


In [ ]:

# Detectando se há algum inteiro e transformando em missing através da função do Numpy np.nan 
cnt=0
for row in data['mfrCode']:
    try:
        int(row)
        data.loc[cnt, 'mfrCode']=np.nan
    except ValueError:
        pass
    cnt+=1




<p>o comando try e except funcionam da seguinte maneira:
o try tentará rodar algum bloco de comando, caso aconteça algum erro, ele roda o bloco de comando do except</p>
<p>nesse caso, estou olhando uma variável que tem apenas letras e estou vendo se algum elemento dela eu consigo transformar em inteiro (o que nesse caso será transformado em NaN). Caso ele não consiga transformar em inteiro, será retornado um erro e o processo seguirá para o except, aonde o comando irá continuar o processo do for normalmente, sem parar por causa do erro,</p>



<p>Perceba que algumas células do nosso dataframe contém <b>NaN</b>, que significa <i>not a number</i>, o que é apenas uma forma
de dizer que a não se tem aquela informação, o dado está ausente.</p>
<p>A função <i>isnull()</i> retorna um df de variáveis booleanas indicando se a informação está ausente</p>


In [ ]:

data.isnull().head()



In [ ]:

# Para saber a quantidade de células com Nan em cada uma das colunas, podemos fazer:
data.isnull().sum()

# Para verificar quantas colunas possuem pelo menos um elemento NaN
len(data.loc[:,data.isnull().sum() > 0].columns)

# Para saber a quantidade de células com NaN apenas nas colunas com NaN:
data.loc[:,data.isnull().sum() > 0].isnull().sum()

# esta primeira parte df.loc[:,df.isnull().sum() > 0] está filtrando a base,
# selecionando todas as linhas (com o comando : no primeiro espaço entre os colchetes) das colunas com pelo menos um NaN
# A segunda parte .isnull().sum() realiza a contagem de quantos NaN tem



In [ ]:

# Para saber a porcentagem de células com Nan em cada uma das colunas, podemos fazer:
data.isnull().sum()/data.shape[0]

# Para verificar quantas colunas possuem pelo menos 80% dos valores como um elemento NaN
len(data.loc[:,data.isnull().sum()/data.shape[0] > 0.8].columns)

# Para saber a quantidade de células com NaN apenas nas colunas com pelo menos 80% de NaN:
data.loc[:,data.isnull().sum()/data.shape[0] > 0.8].isnull().sum()



In [ ]:

# Só para facilitar a visualização dos dados ausentes - para saber mais leia o tutorial 'Seaborn'
# Em preto, os dados ausentes

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(10,5))
sns.heatmap(data.isnull(), cbar=False, yticklabels=False)




<p>E agora? Temos algumas colunas com falta de informações. 
O que fazer? Como resolver esse problema?</p>
<p>Não existe uma maneira correta ou única para resolver isso, vai depender de cada caso. Algumas opções são:</p>
<p>a) Apagar a coluna;    b) Apagar a linha;   c) Preencher com algum valor</p>


In [ ]:

# Vamos resolver o problema da guzzler. Nessa coluna temos 35562 valores nulos. Vimos que nosso dataframe tem 37843 linhas,
# ou seja, aqui não dá pra fazer muita coisa, melhor apagar toda coluna:
data['guzzler2'] = data['guzzler']
data.drop('guzzler2', axis=1, inplace = True) # axis é o eixo: 0 p linha e 1 p coluna
                                         # lembre sempre do inplace = True, para que a mudança seja permanente no nosso df



In [ ]:

# Mas ainda temos muito missings, podemos remover direto as colunas com mais de 10000 valores missings
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

drop_cols = list(null_cols[null_cols > 10000].index)
data = data.drop(drop_cols, axis=1)



In [ ]:

# Agora vamos para a coluna de 'cylinders', uma opção aqui seria preencher os valores ausentes com a média dela

data['cylinders'].mean()



In [ ]:

# Uma solução mais elegante e mais correta seria pegar a média de cada classe de 'Model' para preencher os
# valores, posso fazer isso com o groupby
pd.options.display.float_format = '{:.2f}'.format
data.groupby('VClass')['cylinders'].mean()



In [ ]:

# Agora é só criar uma função para preencher os valores que faltam com as médias calculadas:

def preenchedor(cols):   # a função será aplicada em uma lista de colunas
    cylinders = cols[0]     # sendo que a primeira é a da cylinders, index =[0]
    VClass       = cols[1]     # e a segunda é a da RAT_BACEN, index = [1]
    
    if pd.isnull(cylinders):     # se o valor na coluna cylinders for NaN
        if VClass   == 'Compact Cars':      # e se pertencer a classe "Compact Cars" de VClass
            return 4.82        # retorna a média de cylinders da classe "Compact Cars"  = 4.82
        elif VClass == 'Large Cars':
            return 7.13
        elif VClass == 'Midsize Cars':
            return 5.68  
        elif VClass == 'Midsize Station Wagons':
            return 5.12  
        elif VClass == 'Midsize-Large Station Wagons':
            return 5.18
        elif VClass == 'Minicompact Cars':
            return 5.76
        elif VClass == 'Minivan - 2WD':
            return 5.80 
        else:
            return 0            
    else:                                  # mas se o valor de cylinders não for Nan  
        return cylinders             # recebe o próprio valor
    





In [ ]:

# Aplicando a função para modificar a coluna Age

data['cylinders_2'] = data[['cylinders','VClass']].apply(preenchedor, axis=1)
print("Variável original:  ", data['cylinders'].isnull().sum())
print("VAriável modificada:", data['cylinders_2'].isnull().sum())



In [ ]:

sns.heatmap(data.isnull(), cbar=False, yticklabels=False)



<h2 id="Baixa variância:-"><u>Baixa Variância: </u><a class="anchor-link" href="#Baixa Variância:-">¶</a></h2>

In [ ]:
low_variance = []

for col in data._get_numeric_data():
    minimum = min(data[col])
    ninety_perc = np.percentile(data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)

print(low_variance)

In [ ]:
data = data.drop(low_variance, axis=1)



<h2 id="Outliers:-"><u>Outliers: </u><a class="anchor-link" href="#Outliers:-">¶</a></h2>

In [ ]:
stats = data.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

In [ ]:
outliers = pd.DataFrame(columns=data.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] < lower) | 
                   (data[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)


<h1 id="JUNTAR-DATAFRAMES">JUNTAR DATAFRAMES<a class="anchor-link" href="#JUNTAR-DATAFRAMES">¶</a></h1>



<p>As três principais funções para unir dataframes são: <i>  .concat(), .join() e .merge()</i><br/>
Vamos ver as diferenças entre elas e quando usar cada uma.</p>


In [ ]:

# Temos 2 dataframes df1 e df2

df1 = pd.DataFrame(columns= 'A B C'.split(), index= [0,1,2], data=[['a1','b1','c1'],
                                                                   ['a2', 'b2', 'c2'], 
                                                                   ['a3', 'b3', 'c3']])
df2 = pd.DataFrame(columns= 'A B C'.split(), index= [3,4,5], data=[['a3','b3','c3'],
                                                                   ['a4', 'b4', 'c4'], 
                                                                   ['a5', 'b5', 'c5']])

df3 = pd.DataFrame(columns= 'D E F'.split(), index= [0, 1, 2], data=[['d1','e1','f1'],
                                                                   ['d2', 'e2', 'f2'], 
                                                                   ['d3', 'e3', 'f3']])



In [ ]:

df1



In [ ]:

df2




<p><b> CONCAT ( ) : </b>  basicamente cola dois ou mais dataframes juntos</p>


In [ ]:

# A primeira função que veremos é a 'concat'. Ela basicamente junta dois dataframes.

pd.concat([df1, df2])  # passamos uma lista com os df a serem concatenados



In [ ]:

# Para juntar pelo eixo das colunas, use 'axis=1'. No nosso caso, resultou em alguns valores NaN, pois os df originais
# não possuem a informação das respectivas células

pd.concat([df1, df2], axis=1)



In [ ]:

# Um bom uso para o 'axis=1' seria na concatenação do df1 com o df3, pois compartilham o mesmo índice com colunas diferentes

pd.concat([df1, df3], axis=1)




<p><b> JOIN( ): </b> faz a união de dataframes no índice ou em alguma coluna (transformado em índice)</p>


In [ ]:
data.head()



In [ ]:

# Suponha que você tenha 2 df com informações sobre alguns clientes

func1 = data.loc[1:1000, ['cylinders','drive']]
func2 = data.loc[1:2000, ['trany'      ,'UCity']]




In [ ]:

func1   # func1 tem dados dos clientes sobre o Cylinders e Drive



In [ ]:

func2  # func2 tem dados sobre o trany e UCity



In [ ]:

# O parâmetro 'how' é a forma de união, são quatro opções: left, right, inner, outer 

func1.join(func2)   # left é o padrão: vai usar o índice do df da esquerda, no nosso caso func1



In [ ]:

func1.join(func2, how='right')   # right usa o índice do df da direita, no nosso caso func2



In [ ]:

func1.join(func2, how='inner')  # inner só usa os índices em comum



In [ ]:

func1.join(func2, how='outer')  # outter usa todos os índices




<p>Com esse tutorial rápido já dá pra fazer muita coisa em termo de união de dataframes. <br/>
Para mais informações sobre merge, join e concat, leia a documentação: <a href="http://pandas.pydata.org/pandas-docs/stable/merging.html">http://pandas.pydata.org/pandas-docs/stable/merging.html</a></p>



<h3 id="Aplicando-funções-e-Operações-nos-dataframes:">Aplicando funções e Operações nos dataframes:<a class="anchor-link" href="#Aplicando-funções-e-Operações-nos-dataframes:">¶</a></h3>


In [ ]:

# Primeiro passo é escrever uma função

def dobrar(x):      # função simples que retorna o dobro do número passado
    return x * 2



In [ ]:

# Agora é só aplicar na coluna desejada com .apply(), passando a nossa função como argumento

data['cylinders_2'] = data['cylinders'].apply(dobrar)    # para tornar a alteração permanente, atribuímos o resultado a nossa coluna



In [ ]:

data[['cylinders_2','cylinders']]



In [ ]:

# O mesmo resultado pode ser obtido através de uma função lambda

data['cylinders'].apply(lambda x: x*2)




<h1 id="Normalização-e-Padronização-dos-Dados">Normalização e Padronização dos Dados<a class="anchor-link" href="#Normalização-e-Padronização-dos-Dados">¶</a></h1>



<p>O processo de normalização se dá quando pegamos cada elemento de uma variável e fazemos a diferença com a média e em seguida dividimos pelo desvio padrão. Para realizar esse processo pelo pandas podemos fazer da seguinte maneira:</p>


In [ ]:

data2 = data.loc[:,data.select_dtypes(include = ['float64','int64']).columns].copy()
normalizado_df=(data2 - data2.mean())/data2.std()
normalizado_df.head()



In [ ]:

lista_to_normalize = ['barrels08','city08','city08U', 'co2','co2TailpipeGpm', 'comb08','comb08U', 'cylinders']
data2 = data[lista_to_normalize].copy()
normalizado_df=(data2 - data2.mean())/data2.std()
normalizado_df.head()



In [ ]:

data2 = data.copy()
for coluna in lista_to_normalize:
    data2[coluna] = (data[coluna] - data[coluna].mean()) / (data[coluna].std())




<p>O processo de padronização é quando queremos deixar todos os valores de todas as colunas numéricas entre 0 e 1. Existem diversas técnicas que não funcionam muito bem quando a amplitude das variáveis são muito distintas~, fazendo com que esse processo seja muito útil. Para fazer essa transformação devemos fazer uma subtração entre o valor da variável e o mínimo dela e depois dividir pela amplitude (máximo - mínimo):</p>


In [ ]:

data3 = data.loc[:,data.select_dtypes(include = ['float64','int64']).columns].copy()
padronizado_df=(data3 - data3.min())/(data3.max() - data3.min())
padronizado_df.head()



In [ ]:

lista_to_padronize = ['barrels08','city08','city08U', 'co2','co2TailpipeGpm', 'comb08','comb08U', 'cylinders']
data3 = data.copy()
for coluna in lista_to_padronize:
    df3[coluna] = (data[coluna] - data[coluna].min()) /(data[coluna].max() - data[coluna].min())




<h1 id="Criacao-de-Dummys">Criacao de Dummys<a class="anchor-link" href="#Criacao-de-Dummys">¶</a></h1>



<p>Para criação de dummys no banco de dados, podemos utilizar a função do pandas pd.get_dummies()</p>


In [ ]:

pd.get_dummies(data = data['drive'], drop_first = True, prefix = 'drive',dtype = 'int').head()



In [ ]:

pd.get_dummies(data = data[['drive','trany']], drop_first = True, prefix = ['drive','trany'],dtype = 'int').head()




<h1 id="Gráficos-com-Pandas-e-Matplotlib">Gráficos com Pandas e Matplotlib<a class="anchor-link" href="#Gráficos-com-Pandas-e-Matplotlib">¶</a></h1>



<p>O pandas possuí também uma integração com o matplotlib que facilita muito na construção de gráficos</p>


In [ ]:

# Importar o matplotlib e chama-lo de plt
import matplotlib.pyplot as plt




<p>Para construir um gráfico de barras podemos utilizar a ação ".plot(kind = 'bar')" e devemos ter um objeto do formato "Series" que tenha categorias</p>


In [ ]:

data['drive'].value_counts()



In [ ]:

# Realizar um gráfico de barras
data['drive'].value_counts().plot(kind='bar', color = 'black')




<p>Para fazer um scatterplot devemos realizar a seguinte ação em um pd.DataFrame.plot(x = var1, y = var2, kind = 'scatter)</p>


In [ ]:

data[['UHighway','UCity']].plot(x ='UHighway',  y = 'UCity',   kind='scatter', color = 'black')



In [ ]:
data[['UHighway','UCity']].plot(x ='UHighway'
                                             ,  y = 'UCity'
                                             ,   kind='scatter'
                                             , color = 'black'
                                            ,xlim = [0,100]
                                            ,ylim = [0,100])




<p><b>FONTES:</b></p>
<p>1 - <a href="http://pandas.pydata.org/">http://pandas.pydata.org/</a> <br/>
2 - <a href="http://pandas.pydata.org/pandas-docs/stable/">http://pandas.pydata.org/pandas-docs/stable/</a><br/>
3 - <a href="http://pandas.pydata.org/pandas-docs/stable/10min.html#min">http://pandas.pydata.org/pandas-docs/stable/10min.html#min</a><br/>
4 - <a href="http://pandas.pydata.org/pandas-docs/stable/api.html#general-functions">http://pandas.pydata.org/pandas-docs/stable/api.html#general-functions</a><br/></p>
